<a href="https://colab.research.google.com/github/qhfks27/rani01/blob/master/CH03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#정책 그래디언트  

>강화학습 문제 해결방법은 정책 실행을 통한 샘플(데이터)생성, 모델 또는 가치함수의 추정, 정책개선 등 3단계로 구성되는 반복(이터레이션, iteration) 과정을 통해 환경으로부터 받는 누적 보상을 최대화하는 최적의 정책을 산출하는 것이다.
  
 그 중 정책 그래디언트는 직접 정책을 유도하는 방법이다. 반환값의 기댓값으로 이루어진 목적함수 J를 최대로 만드는 정책 을 최적화하는 정책 파라미터θ를 계산하는 것이 목표이다. 보통 가치함수도 함께 추정하여 정책의 성과를 평가하는 액터-크리틱(actor-critic)구조를 사용한다. 이 방법은 먼저 정책을 파라미터화 하고, 누적보상을 정책의 함수로 기술한 후, 최적화 방법으로 누적보상을 최대화하는 정책 파라미터를 계산한다.
$$\theta^{*} =argmaxJ(\theta)$$
$$J(\theta)=E_{\tau~ p_{\theta}}[\sum_{t=0}^T \gamma^{t} r(x_t, u_t)]$$

는 시간스텝 t일 때 상태변수 에서 행동를 사용했을 때 에이전트가 받는 보상, γ는 감가율(discount factor), τ는 정책 로 생성되는 궤적이다. 정책을 파라미터화하는 신경망을 정책신경망(policy neural network)라고 하고 θ는 신경망의 모든 가중치이다.  
$$G_0 = \sum_{t=0}^T \gamma^{t} r(x_t, u_t) = \sum_{k=t}^T \gamma^{k-t} r(x_k, u_k) $$

전체반환값 는 t=0부터 에피소드가 종료될 때까지 받을 수 있는 전체 궤적에 대한 감가보상(discount reward)의 총합이다. 는 임의의 시간 부터 에피소드가 종료될 때까지 받을 수 있는 예정보상(reward-to-go)의 총합이다.
$$p_\theta(\tau)=p_\theta(x_0.u_0,x_1,u_1,...,x_T,u_T)$$  
위 식을 베이즈 정리와, 마르코프 시퀀스 가정에 의해 전개하면 아래와 같은 식을 얻을 수 있다.
$$p_\theta(\tau)=p(x_0)\prod_{t=0}^T \pi_\theta(u_t|x_t)p(x_{t+1}|x_t,u_t)$$  
  
  목적함수는 상태가치 함수와도 관계가 있으므로 아래의 식을 얻을 수 있다.  
  $$J(\theta)=E_{\tau~ p_{\theta}}[\sum_{t=0}^T \gamma^{t} r(x_t, u_t)]
  =\int_{\tau} p_{\theta}(\tau) (\sum_{t=0}^T \gamma^{t} r(x_t, u_t)) d\tau = \int_{x_0}[\int_{\tau_{u_0:u_t}}p_{\theta}(\tau_{u_0:u_T}|x_0)(\sum_{t=0}^T \gamma^t r(x_t, u_t))d\tau_{u_0:u_T}]p(x_0)dx_0 = \int_{x_0} V^{\pi_\theta}(x_0)p(x_0)dx_0 = E_{x_0 ~ p(x_0)}[V^\pi_\theta(x_0)]$$



베이즈 정리와 상태가치함수 를 대입하면 목적함수가 초기 상태변수 에 대한 상태가치의 평균값이 됨을 확인할 수 있다.
 목적함수 를 최대로 만들기 위해 목적함수를 θ로 미분해보면 아래의 식을 얻을 수 있다.
$${\partial J(\theta) \over \partial\theta} = \nabla_\theta J(\theta) = \int_{\tau} \nabla_\theta p_\theta (\tau) \sum_{t=0} ^T \gamma^t r(x_t,u_t) d\tau = E_{\tau ~ p_\theta (\tau)}[\sum_{t=0}^T \nabla_\theta \log\pi_\theta (u_t|x_t))(\sum_{t=0}^T \gamma^{k-t}r(x_t,u_t))]$$
위의 식에서 가 목적함수 그래디언트 식에서 사라짐으로써 환경모델이 필요 없는 모델프리(model-free) 강화학습 방법이 된다.


$$\nabla_\theta J(\theta)= E_{\tau ~ p_\theta (\tau)}[\sum_{t=0}^T \nabla_\theta \log\pi_\theta (u_t|x_t))(\sum_{t=0}^T \gamma^{k-t}r(x_t,u_t))]$$  
위 식은 예상 보상에만 감가율 적용하여 당초 목적함수의 편향된 그래디언트이다.
$$\theta \leftarrow \theta + \alpha \nabla_\theta J(\theta)$$
목적함수를 최대로 하는 파라미터θ는 위의 경사상승법으로 구할 수 있다. 즉, 정책 그래디언트는 목적함수의 그래디언트를 이용해 정책을 업데이트하는 방법이다.




##REINFORCE 알고리즘
τ상의 기댓값 는 수학적으로 직접계산 할 수 없으므로 어떤 정책을 실제로 실행해서 나온 에피소드인 샘플을 이용해 추정한다.  
$$\nabla_\theta J(\theta)\approx {1 \over M} \sum_{m=1} ^M[\sum_{t=0}^T \log \pi_\theta(u_t^{(m)}|x_t^{(m)})(\sum_{k=t}^T \gamma^{k-t}r(x_t^{(m)},u_t^{(m)}))]=\nabla_\theta{1 \over M} \sum_{m=1} ^M[\sum_{t=0}^T \log \pi_\theta(u_t^{(m)}|x_t^{(m)})G_t^{(m)})]$$
위 식으로 에피소드를 M개만큼 샘플링 해 에피소드 평균을 이용해 근사적으로 계산한다. 아래의 loss는 에피소드의 손실함수이다.
$$\theta \leftarrow \theta + \alpha \nabla_\theta J(\theta) \approx \theta - \alpha \nabla_\theta \sum_{t=0}^T\log \pi_\theta(u_t^{(m)}|x_t^{(m)})G_t^{(m)})]$$  
$$loss=-\sum_{t=0}^T\log \pi_\theta(u_t^{(m)}|x_t^{(m)})G_t^{(m)})]$$


신경망은 손실함수를 최소화하도록 파라미터가 업그레이드되고, 정책 그래디언트는 목적함수를 최대로 해야 한다.  
즉, REINFORCE 알고리즘은 정책으로부터 에피소드를 생성하고 이 에피소드의 반환값과 손실함수를 계산한다. 그 후 정책 파라미터를 업데이트함으로써 정책을 업데이트 하는 방법이다. 이 알고리즘은 에피소드에 따라 학습시간이 길어지고 그래디언트의 분산이 매우 커 현재 사용하지 않지만 다른 정책 그래디언트의 기반 알고리즘의 기초가 되기 때문에 의의를 갖는다.
